This notebook follows on from my first attempt at using geo data in folium, by exploring the use of Bokeh to make interactive choropleth maps. <br/>
The bokeh docs were very useful as was this blog for creating the Bokeh plots:<br/>
https://data-dive.com/cologne-bike-rentals-interactive-map-bokeh-dynamic-choropleth<br/>
First import a few libraries.

In [1]:
import folium
import geopandas as gpd
import pandas as pd

print(folium.__version__)
print(pd.__version__)
print(gpd.__version__)

0.5.0
0.20.1
0.3.0


Then read one of the csv files and load it into a dataframe.

In [2]:
# read csv
df_LSOA_2005_2007 = pd.read_csv('accidents_2005_to_2007.csv',
                                usecols=['Year','LSOA_of_Accident_Location'],low_memory=False)
# select data for 2005
df_LSOA_2005 = pd.DataFrame(df_LSOA_2005_2007[df_LSOA_2005_2007['Year']==2005])
# rename column to use for merging later. LSOA is a postcode location within the UK. 
df_LSOA_2005.rename(columns={'LSOA_of_Accident_Location':'LSOA11CD'},inplace=True)

Result is a Data Frame with every LSOA (Lower Super Output Layer) that had an accident recorded in the csv for the year 2005.

In [3]:
print(len(df_LSOA_2005))
df_LSOA_2005.head()

198735


,LSOA11CD,Year
0,E01002849,2005
1,E01002909,2005
2,E01002857,2005
3,E01002840,2005
4,E01002863,2005


The same issue as I encounted in the previous notebook will need to resolved here too. The csv files use the LSOA whereas the geojson uses LAD (Local Authority District).

In [25]:
print(len(df_LSOA_2005['LSOA11CD'].value_counts()))

28396


By loading the geojson into geopandas we can see there are only 380 LADs compared to 28396 LSOAs

In [5]:
geo_json = 'Local_Authority_Districts_Dec_2016.geojson'
gdf = gpd.read_file(geo_json)
print(len(gdf))
gdf.head()

380


,objectid,lad16cd,lad16nm,lad16nmw,bng_e,bng_n,long,lat,st_areashape,st_lengthshape,geometry
0,1,E06000001,Hartlepool,,447157,531476,-1.27023,54.676159,9.355951e+07,71707.330231,(POLYGON ((-1.268455585200569 54.7261163502866...
1,2,E06000002,Middlesbrough,,451141,516887,-1.21099,54.544670,5.388858e+07,43840.846371,"(POLYGON ((-1.2439036468982 54.58936316454281,..."
2,3,E06000003,Redcar and Cleveland,,464359,519597,-1.00611,54.567520,2.448203e+08,97993.287164,(POLYGON ((-1.137578154500661 54.6458090223704...
3,4,E06000004,Stockton-on-Tees,,444937,518183,-1.30669,54.556911,2.049622e+08,119581.507757,(POLYGON ((-1.317285862199188 54.6448033247461...
4,5,E06000005,Darlington,,428029,515649,-1.56835,54.535351,1.974757e+08,107206.282926,POLYGON ((-1.637677653638193 54.61713779015086...


My initial thought was to loop through the Data Frame and assign each coordinate to an area.

In [6]:
import json
from shapely.geometry import shape, Point
with open(geo_json) as f:
    data = json.load(f)

In [7]:
%%time
# construct point based on lon/lat returned by geocoder
point = Point(-0.164002,51.484087)

# check each polygon to see if it contains the point
for feature in data['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        print ('Found')
    

Found
Wall time: 10.9 s


On my PC it took anywhere between 8-10s per point, so for 154414 points this would take around 16 days to complete! <br/>
However there is a lookup table between LSOA and LAD at: <br/> http://geoportal.statistics.gov.uk/datasets/3ecc1f604e0148fab8ea0b007dee4d2e_0

In [8]:
df_LAD = pd.read_csv('Output_Area_to_LAD_to_LLSOA_to_MLSOA_to_LEP_April_2017_Lookup_in_England_V2.csv',
                       usecols=['LAD16CD','LSOA11CD'])
df_LAD.head()

,LAD16CD,LSOA11CD
0,E08000031,E01010462
1,E08000031,E01010449
2,E08000031,E01010420
3,E08000031,E01010485
4,E08000031,E01010475


df_LAD contains all the LSOA locations within each LAD, it is then easy to merge the two Data Frames and count all the accidents within each LAD.

In [9]:
#Merge Data Frames
df_LSOA_2005 = df_LSOA_2005.merge(df_LAD,on='LSOA11CD')
#Create df_choro which the count of all the accidents within each LAD in 2005.
df_choro = pd.DataFrame(df_LSOA_2005['LAD16CD'].value_counts().reset_index())
df_choro.rename(columns={'index':'LAD16CD','LAD16CD':'Count'},inplace=True)
print (len(df_choro))
df_choro.head()

326


,LAD16CD,Count
0,E08000025,17440
1,E08000035,11815
2,E06000052,11799
3,E08000003,9981
4,E08000012,9919


Now to import all the libraries required for Bokeh.

In [10]:
from bokeh.io import show, output_notebook, output_file
from bokeh.models import (
    GeoJSONDataSource,
    HoverTool,
    LinearColorMapper,
    ColumnDataSource
)
from bokeh.plotting import figure
from bokeh.palettes import Viridis6
import json
output_notebook()

Loading BokehJS ...

The plotting in Bokeh requires the geometries to be in x and y coordinate to plot correctly, and polygons and multipolygons need to be handled correctly to allow accurate generation of the map. This great function reads the geojson, converts the geometries into the correct format and returns a Data Frame. This can subsequently be used as the basemap for plotting our Bokeh maps.

In [11]:
#http://nbviewer.jupyter.org/urls/gist.githubusercontent.com/RutgerK/e707239e1f82b3287a5d/raw/Bokeh_test
import ogr

def getAttributes(inds):
    # read the attributes of a vector (OGR) dataset and
    # returns it as a Pandas DataFrame, including the
    # geometry, which is formatted for Bokeh (xs, ys)
    
    def getFielddef(lyr):
        # function to get the field definitions
        # of the attributes
        
        lyrdef = lyr.GetLayerDefn()
        
        fielddef = {}
        
        for i in range(lyrdef.GetFieldCount()):
            fdef = lyrdef.GetFieldDefn(i)
            fielddef[fdef.GetName()] = (i, fdef.GetTypeName())
            
        return fielddef
    
    # open dataset and layer
    ds = ogr.Open(inds)
    lyr = ds.GetLayer(0)
    
    lyr.ResetReading()
    ft = lyr.GetNextFeature()

    features = []
    
    fielddef = getFielddef(lyr)

    # loop over all features
    while ft:
        
        ft_atts = {}

        # read all atributes for the feature
        for att_name, (i, att_type) in fielddef.items():
            if att_type == 'String':
                ft_atts[att_name] = ft.GetFieldAsString(att_name)
            elif att_type == 'Integer':
                ft_atts[att_name] = ft.GetFieldAsInteger(att_name)
            elif att_type == 'Real':
                ft_atts[att_name] = ft.GetFieldAsDouble(att_name)
            elif att_type == 'Date':
                ft_atts[att_name] = pd.datetools.parse(ft.GetField(i))
            elif att_type == 'DateTime':
                ft_atts[att_name] = pd.datetools.parse(ft.GetFieldAsDateTime(att_name)).to_datetime()

        # get the geometry
        geom = ft.GetGeometryRef()

        geom_json = json.loads(geom.ExportToJson())

        geom_type = geom.GetGeometryType()

        if geom_type == ogr.wkbMultiPolygon:
            # append NaN after each part of a multipolygon
            ys = [list(zip(*sum(poly, [])))[1] for poly in geom_json['coordinates']]
            ys = sum([list(part) + [float('NaN')] for part in ys], [])

            xs = [list(zip(*sum(poly, [])))[0] for poly in geom_json['coordinates']]
            xs = sum([list(part) + [float('NaN')] for part in xs], [])

        elif geom_type == ogr.wkbPolygon:
            denest = list(zip(*sum(geom_json['coordinates'], [])))

            ys = denest[1]
            xs = denest[0]

        ft_atts['xs'] = xs
        ft_atts['ys'] = ys

        sr = pd.Series(ft_atts)
        sr.name = ft.GetFID()
        features.append(sr)

        ft = lyr.GetNextFeature()

    attributes = pd.concat(features, axis=1).T
    attributes.index.name = 'FID'
    
    ds = None

    return attributes

We then create a Data Frame with all the LAD within Greater London, which correspond to all the London Boroughs (plus the city of London).

In [12]:
df_boroughs = pd.read_csv('Local_Authority_District_to_Region_December_2016_Lookup_in_England.csv',usecols=['LAD16CD','LAD16NM','RGN16NM'])
df_boroughs = df_boroughs[df_boroughs['RGN16NM']=='London']
print(len(df_boroughs))
df_boroughs.head()

33


,LAD16CD,LAD16NM,RGN16NM
40,E09000001,City of London,London
49,E09000002,Barking and Dagenham,London
52,E09000003,Barnet,London
60,E09000004,Bexley,London
65,E09000005,Brent,London


This Data Frame can then be merged with df_Choro, which contains the counts of accidents in each LAD for 2005, giving us a Data Frame with each LAD within Greater London and the number of accidents occuring in 2005.

In [13]:
df_choro = df_choro.merge(df_boroughs,on='LAD16CD')
df_choro.rename(columns={'LAD16CD':'lad16cd'},inplace=True)
df_choro.sort_values(by='lad16cd',inplace=True)
print(len(df_choro))
df_choro.head()

33


,lad16cd,Count,LAD16NM,RGN16NM
32,E09000001,124,City of London,London
27,E09000002,2694,Barking and Dagenham,London
4,E09000003,5207,Barnet,London
26,E09000004,2730,Bexley,London
11,E09000005,4625,Brent,London


Now the getAttributes function is used to convert the geojson into a Data Frame that Bokeh can use to plot. There are a number of columns that the function returns that can be dropped, as we will only be concerned with the LAD unique identifier and the xs and ys coordinates.

In [53]:
infile = r'Local_Authority_Districts_Dec_2016.geojson'
df_points = getAttributes(infile)
df_points.drop(['bng_e','bng_n','lat','long','objectid','st_areashape','st_lengthshape','lad16nmw','lad16nm'],axis=1,inplace=True)
print(len(df_points))
df_points.head()

380


,lad16cd,xs,ys
FID,,,
0,E06000001,"[-1.268455585200569, -1.268224717407081, -1.26...","[54.72611635028668, 54.72609061215454, 54.7261..."
1,E06000002,"[-1.2439036468982, -1.242572165669934, -1.2424...","[54.58936316454281, 54.58864311230379, 54.5886..."
2,E06000003,"[-1.137578154500661, -1.137808310275346, -1.13...","[54.64580902237043, 54.645112308700575, 54.644..."
3,E06000004,"[-1.317285862199188, -1.317148076663826, -1.31...","[54.64480332474616, 54.64479895504185, 54.6448..."
4,E06000005,"(-1.637677653638193, -1.637672166576906, -1.63...","(54.61713779015086, 54.61670374895607, 54.6163..."


In [58]:
df_points.to_csv("LAD16CD_xs_ys.csv",index=False)

In [59]:
df_points2 = pd.read_csv("LAD16CD_xs_ys.csv")

In [60]:
df_points2.head()

,lad16cd,xs,ys
0,E06000001,"[-1.268455585200569, -1.268224717407081, -1.26...","[54.72611635028668, 54.72609061215454, 54.7261..."
1,E06000002,"[-1.2439036468982, -1.242572165669934, -1.2424...","[54.58936316454281, 54.58864311230379, 54.5886..."
2,E06000003,"[-1.137578154500661, -1.137808310275346, -1.13...","[54.64580902237043, 54.645112308700575, 54.644..."
3,E06000004,"[-1.317285862199188, -1.317148076663826, -1.31...","[54.64480332474616, 54.64479895504185, 54.6448..."
4,E06000005,"(-1.637677653638193, -1.637672166576906, -1.63...","(54.61713779015086, 54.61670374895607, 54.6163..."


Merging the Data Frame containing all the geometries of each LAD and the counts of all the accidents occuring in that LAD will provide a Data Frame with all the information required for Bokeh to create a choropleth.

In [51]:
df_data = pd.merge(df_points,df_choro,on='lad16cd')
df_data.sort_values('lad16cd',inplace=True)
print(len(df_data))
df_data.head()

33


,FID,lad16cd,xs,ys,Count,LAD16NM,RGN16NM
0,293,E09000001,"(-0.096687783724441, -0.096671641368652, -0.09...","(51.52319420249525, 51.52316517178213, 51.5231...",124,City of London,London
1,294,E09000002,"[0.148097769041267, 0.148086753569273, 0.14979...","[51.59640478014033, 51.59635553399191, 51.5970...",2694,Barking and Dagenham,London
2,295,E09000003,"(-0.199871677368659, -0.199675526311897, -0.19...","(51.670170272803766, 51.66986331938452, 51.669...",5207,Barnet,London
3,296,E09000004,"[0.128620874554508, 0.130409423392165, 0.13072...","[51.513294145134424, 51.512951389126926, 51.51...",2730,Bexley,London
6,299,E09000005,"(-0.264736981582663, -0.264251203474209, -0.26...","(51.598184934557295, 51.59778293838356, 51.597...",4625,Brent,London


The following cell is all the code required to produce the interactive choropleth in Bokeh. Essentially the data to use is supplied as a list of lists, any additional tools are defined

In [16]:
# source data as a list of lists
source = ColumnDataSource(data=dict(xs=df_data['xs'].tolist(),ys=df_data['ys'].tolist(),
    Borough=df_data['LAD16NM'],
    Count=df_data['Count'],
))

# palette to use
color_mapper = LinearColorMapper(palette=Viridis6)

# tools to load
TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

# create a new figure for plotting
p = figure(title="Accidents within London Boroughs occuring in 2005.", tools=TOOLS, x_axis_location=None, y_axis_location=None,height=550,width=750)
p.grid.grid_line_color = None

#patches - creates the 2D polygons with a face colour and an edge colour. The edge colour tends to be consistent, and the face
#colour depends on the fill_colour value passed to it, therefore allowing for different colours based on differing values.
p.patches('xs', 'ys',source=source, fill_alpha=0.7, fill_color={'field': 'Count', 'transform': color_mapper}, 
          line_color='white', line_width=0.5)

# define type of mouse hover and the data to display passed as a list of tuples.
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@Borough"),("Number of Accidents","@Count")]

output_file("Accidents within London Boroughs occuring in 2005.html", title="London Borough Choropleth test")

show(p)

A nice interactive choropleth is then created, however it could be nice to see how the number of accidents change over the years. Bokeh can do this with a handy slider! <br/>
Firstly lets load all the data from 2005 to 2014 into one Data Frame and merge it with the df_points.

In [17]:
#Load all the csv files into Data Frames
df_LSOA_2005_2007 = pd.read_csv('accidents_2005_to_2007.csv',usecols=['Year','LSOA_of_Accident_Location'],low_memory=False)
df_LSOA_2007_2011 = pd.read_csv('accidents_2009_to_2011.csv',usecols=['Year','LSOA_of_Accident_Location'],low_memory=False)
df_LSOA_2012_2014 = pd.read_csv('accidents_2012_to_2014.csv',usecols=['Year','LSOA_of_Accident_Location'],low_memory=False)
#concatenate into one Data Frame
df_2005_2014 = pd.concat([df_LSOA_2005_2007,df_LSOA_2007_2011,df_LSOA_2012_2014])
#tidy names
df_2005_2014.rename(columns={'LSOA_of_Accident_Location':'LSOA11CD'},inplace=True)
#Merge with df_LAD to get the LAD of each accident.
df_2005_2014 = df_2005_2014.merge(df_LAD,on='LSOA11CD')
print(len(df_2005_2014))
df_2005_2014.head()

7016953


,LSOA11CD,Year,LAD16CD
0,E01002849,2005,E09000020
1,E01002849,2005,E09000020
2,E01002849,2005,E09000020
3,E01002849,2005,E09000020
4,E01002849,2005,E09000020


The Data Frame can then be grouped by year and then all the accidents for each LAD can be counted for a particular year. <br/>

In [18]:
df_2005_2014 = pd.DataFrame(df_2005_2014.groupby(['Year'])['LAD16CD'].value_counts())
#Tidy names
df_2005_2014.rename(columns={'LAD16CD':'Count'},inplace=True)
df_2005_2014.reset_index(inplace=True)
#Merge with df_Boroughs to filter out the data for London.
df_2005_2014 = pd.merge(df_2005_2014,df_boroughs,on='LAD16CD')
#Sort by LAD.
df_2005_2014.sort_values('LAD16CD',inplace=True)
print(len(df_2005_2014))
df_2005_2014.head()

297


,Year,LAD16CD,Count,LAD16NM,RGN16NM
296,2014,E09000001,1484,City of London,London
294,2012,E09000001,223,City of London,London
293,2011,E09000001,192,City of London,London
292,2010,E09000001,170,City of London,London
291,2009,E09000001,170,City of London,London


As a sense check the length of this Data Frame is 297, which is 9 x 33 (No. of years x No. of boroughs). <br/>
The Data Frame is sorted by LAD, as will become evident later, the way the lists of lists are passed into Bokeh the counts of the accidents and the LAD need to be sorted in the same way. Therefore as each year is filtered then the LAD is in the correct order for the df_points Data Frame to marry up the data.<br/>
A dictionary is then created with the year as the key, and a list of the number of accidents as the values.

In [19]:
years = dict()
for i in range(len(df_2005_2014['Year'].unique())):
    year = df_2005_2014['Year'].unique()[i]
    years[str(year)] = df_2005_2014[df_2005_2014['Year']==year]['Count'].tolist()

This dictionary is then merged with the source data used in the previous map. This dictionary then has the xs, ys, Borough names, counts for 2005, and then every year as a key (with the counts embeded within this)

In [20]:
data=dict(xs=df_data['xs'].tolist(),ys=df_data['ys'].tolist(),
    Region=df_data['LAD16NM'],
    Count=df_data['Count'],**years
          )
data.keys()

dict_keys(['xs', 'ys', 'Region', 'Count', '2014', '2012', '2011', '2010', '2009', '2007', '2006', '2005', '2013'])

The map is then set up in much the same way as the previous plot.

In [21]:
source = ColumnDataSource(data=data)

color_mapper = LinearColorMapper(palette=Viridis6)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="London Boroughs", tools=TOOLS, x_axis_location=None, y_axis_location=None,height=550,width=750)
p.grid.grid_line_color = None

p.patches('xs', 'ys',source=source, fill_alpha=0.7, fill_color={'field': 'Count', 'transform': color_mapper}, 
          line_color='white', line_width=0.5)


hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@Borough"),("Number of Accidents","@Count")]

We then need some additional libraries

In [22]:
from bokeh.layouts import column, row, widgetbox
from bokeh.models import CustomJS, Slider, Toggle

#set file name to save.
output_file("Number of accidents 2005-2014.html")

Firstly add a slider to allow different years to be selected, between the minimum and maximum with an interval (step) of 1.

In [23]:
# add slider
slider = Slider(start=2005, end=2014, value=2005, step=1, title="Year")

There then needs to be a mechanism to update the map as the slider is moved. This is performed by a callback function through js_on_change. So when the slider value is changed the update function is called. This function takes the year selected by the slider and updates the data['Count'] with that years values.

In [24]:
def update(source=source, slider=slider, window=None):
    """ Update the map: change the count of accidents according to slider
        will be translated to JavaScript and Called in Browser """
    data = source.data
    v = cb_obj.get('value')
    data['Count'] = [x for x in data[v]]
    source.trigger('change')

slider.js_on_change('value', CustomJS.from_py_func(update))
show(column(p,widgetbox(slider),))